# Google Image Scraper

Hello everyone,

This Jupyter notebook provides a step-by-step guide on how to download images from Google using a Python script. The notebook leverages the `simple_image_download` library (yes, a creative name) to handle the task of fetching and downloading images or obtaining their URLs. Additionally, `matplotlib` is used to display the images.

This script is adapted from Abhay Parashar's amazing article: "17 Mindblowing Python Automation Scripts I Use Everyday."



Recommendations:
  - `simple-image-download 0.5` (Library for downloading images from google)
  -  `Python 3.10` (click on top right corner to change version)
  - `matplotlib 3.4.3` (Library for plotting graphs, but in this case it is used to display images)
  - `pillow` (Library for processing images, in this case: url --> image object --> plot using matplotlib)


# Download the dependencies required for this Jupyter Notebook

These commands will install the necessary packages `simple-image-download` and `matplotlib` that are used in the notebook.

In [ ]:
%pip install simple-image-download
%pip install matplotlib

In [ ]:
# Words or phrases you google images of:
keywords = ['cat', 'dog', 'bird', 'snake']

# Number of images to download for each keyword:
num_images = [ 10, 5, 3, 2]

# Notes: keywords and num_images must be alligned by index and consequently have the same length.

In [ ]:
# Importing the libraries
from simple_image_download import simple_image_download as simp 

# Creating a response object
response = simp.simple_image_download

image_urls = {}

for word, num in zip(keywords, num_images):
    try:
        urls = response().urls(word, num+1) # first image is always the gstatic image
        image_urls[word] = urls[1:]
        print(f"URLs for {word}:")
        for url in urls[1:]:
            i = urls.index(url)
            print(f"\t image{i}:\t {url}")

    ## To download localy the images uncomment the following line
    # response().download(word, num)

    except Exception as e:
        print("An error occurred:", e)

In [ ]:
from PIL import Image, UnidentifiedImageError
import requests
from io import BytesIO
import math
import matplotlib.pyplot as plt
import textwrap

def create_image_array(num):
    return math.ceil(num / 3)

def resize_image(image, target_size=(300, 300)):
    return image.resize(target_size, Image.LANCZOS)

def display_images(image_urls, word):
    num_images = len(image_urls)
    num_rows = create_image_array(num_images)

    fig = plt.figure(figsize=(15, 5 * num_rows))
    plt.suptitle(f"Images of {word}", fontsize=16, y=0.95)

    for index, url in enumerate(image_urls, start=1):
        ax = fig.add_subplot(num_rows, 3, index)
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.axis('off')
        
        try:
            response = requests.get(url)
            response.raise_for_status()
            image = Image.open(BytesIO(response.content))
            
            # Resize the image
            resized_image = resize_image(image)

            # Plotting the image
            ax.imshow(resized_image, extent=[0, 1, 0, 1])
        except UnidentifiedImageError:
            error_message = "Error: Unable to identify image"
            wrapped_text = textwrap.fill(error_message, width=20)
            ax.text(0.5, 0.5, wrapped_text, ha='center', va='center', wrap=True, color='red')
        except requests.RequestException as e:
            error_message = f"Error fetching image: {str(e)}"
            wrapped_text = textwrap.fill(error_message, width=20)
            ax.text(0.5, 0.5, wrapped_text, ha='center', va='center', wrap=True, color='red')
        except Exception as e:
            error_message = f"Unexpected error: {str(e)}"
            wrapped_text = textwrap.fill(error_message, width=20)
            ax.text(0.5, 0.5, wrapped_text, ha='center', va='center', wrap=True, color='red')
        
        ax.set_title(f"Image {index}")

    plt.tight_layout()
    plt.subplots_adjust(top=0.9, hspace=0.5)  # Adjusted top margin and increased vertical space between subplots
    plt.show()


for word, urls in image_urls.items():
    display_images(urls, word)